# Urban Water Model Simulation for Fehraltorf

This notebook runs the urban water model simulation for Fehraltorf and visualizes the results.

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd

from ipywidgets import Button, Output
import ipywidgets as widgets
import plotly.graph_objects as go

import holoviews as hv
from bokeh.io import output_notebook
hv.extension('bokeh')
output_notebook()

from duwcm.read_data import read_data
from duwcm.forcing import read_forcing, distribute_irrigation
from duwcm.water_model import UrbanWaterModel
from duwcm.water_balance import run_water_balance
from duwcm.functions import load_config, select_cells
from duwcm.scenario_manager import ScenarioManager, Scenario, run_scenario
from duwcm.postprocess import extract_local_results

from duwcm.viz import (
    plot_aggregated_results,
    create_map_base,
    create_dynamic_map,
    create_flow_visualization,
    create_reuse_visualization,
    interactive_cell_selection
)

## Configuration

Set up the configuration for the simulation.

In [ ]:
config_path = '.'
config = load_config('.', 'default', 'config.yaml')

print(f"Input directory: {config.input_directory}")
print(f"Output directory: {config.output.directory}")
print(f"Simulation period: {config.simulation.start_year} - {config.simulation.end_year}")

geo_file = Path(config.input_directory) / Path(config.files.geo)
background_file = Path(config.geodata_directory) / config.files.background_shapefile

_, _, _, _, _, flow_paths = read_data(config)

# Call the interactive selection function and store selected cells
selected_cells = interactive_cell_selection(config, geo_file, background_file, flow_paths)

## Run Simulation

Execute the urban water model simulation.

In [ ]:
# Read and prepare model data
model_params, reuse_settings, demand_data, soil_data, et_data, flow_paths = read_data(config)
forcing_data = read_forcing(config)

# Filter selected cells
if selected_cells is not None:
    model_params, flow_paths = select_cells(model_params, flow_paths, selected_cells)

# Create scenario 
default_scenario = Scenario(
    name='default',
    description='Default configuration'
)

# Create model data dict
model_data = {
    'flow_paths': flow_paths, 
    'soil_data': soil_data,
    'et_data': et_data,
    'demand_data': demand_data,
    'reuse_settings': reuse_settings,
    'direction': config.grid.direction
}

In [ ]:
# Create and run scenario
name, results = run_scenario((default_scenario.name, model_params, forcing_data, model_data, False, None, True))

## Visualize Results

Create plots to visualize the simulation results.

In [ ]:
fig = plot_aggregated_results(results['aggregated'], forcing_data)
fig.show()

## Generate Interactive Maps

Create interactive map visualizations of the results.

In [ ]:
area_variables = [
    'evapotranspiration',
    'imported_water',
    'baseflow',
    'deep_seepage',
    'stormwater_runoff',
    'sewerage_discharge',
    'groundwater',
    'vadose_moisture'
]

gdf_geometry = gpd.read_file(geo_file)
local_results = extract_local_results(results)
# Filter results to selected cells if specified
if hasattr(config.grid, 'selected_cells'):
    local_results = local_results[local_results.index.get_level_values('cell').isin(config.grid.selected_cells)]

time_series_data = local_results[area_variables].unstack(level='cell')
time_series_data.attrs['units'] = local_results.attrs['units']

dynamic_map = create_dynamic_map(gdf_geometry, background_file, area_variables, time_series_data)
dynamic_map.show()

## Generate Alluvial diagrams

In [ ]:
sankey_fig = create_flow_visualization(results, viz_type='sankey')
sankey_fig.show()

In [ ]:
sankey_fig = create_reuse_visualization(results, viz_type='sankey')
sankey_fig.show()

## Generate Chord diagrams

In [ ]:
chord_fig = create_flow_visualization(results, viz_type='chord') 
chord_fig

In [ ]:
chord_fig = create_reuse_visualization(results, viz_type='chord') 
chord_fig